In [1]:
#regex for password
#kiwi->spacy ner for name
#sample data:"김민수는 서울대학교에 재학 중이고 계정 비밀번호는 TestAccount2026이다."


In [2]:
import spacy
from kiwipiepy import Kiwi 
import re

In [3]:
nlp = spacy.load("ko_core_news_lg")

In [4]:
kiwi = Kiwi()

In [5]:
#SENSITIVE_KEYWORD = ["비밀번호"]

In [6]:
PASSWORD_REGEX = r"\b[A-Za-z0-9@#$%^&+=!]{8,}"

In [7]:
def is_sensitive_context(text):
    return any(k in text for k in SENSITIVE_KEYWORD)

In [8]:
#tokennize text on kiwi
text = "김민수는 서울대학교에 재학 중이고 계정 비밀번호는 TestAccount2026이다."
tokens = kiwi.tokenize(text)

In [9]:
#kiwi processing
def kiwi_preprocess(text):
    tokens = kiwi.tokenize(text)
    return " ".join([t.form for t in tokens])

In [10]:
#detect spacy in kiwi tokennize text
clean_text = kiwi_preprocess(text)
doc = nlp(clean_text)

In [11]:
#see ner in doc 
for ent in doc.ents:
    print(ent.text, ent.label_)

김민수 PS
서울대 OG
2026 QT


In [12]:
#extract regex target
def extract_regex(text):
    regex_target = []
    regex_target +=re.findall(PASSWORD_REGEX, text)
    return regex_target

In [13]:
#extract entity target
def extract_ner(text):
    doc = nlp(text)
    return [
        ent.text
        for ent in doc.ents
        if ent.label_ in {"PS", "OG"}
    ]

In [14]:
#combine regex found pattern and ner in doc and
#mask it on the original doc

In [15]:
#mask data
def mask_text(text, targets):
    masked_text = text
    for t in set(targets):
        masked_text = masked_text.replace(t, "*" * len(t))
    return masked_text

In [16]:
text = "김민수는 서울대학교에 재학 중이고 계정 비밀번호는 TestAccount2026이다."

clean_text = kiwi_preprocess(text)
print(clean_text)

ner_target = extract_ner(clean_text)
print(ner_target)

#if not is_sensitive_context(text):
#    text, False

re_target = extract_regex(text)
print(re_target)

allt = re_target + ner_target
masked = mask_text(text, allt)
print(text)
print(allt)

print(masked)

김민수 는 서울대 학교 에 재학 중 이 고 계정 비밀번호 는 TestAccount 2026 이 다 .
['김민수', '서울대']
['TestAccount2026']
김민수는 서울대학교에 재학 중이고 계정 비밀번호는 TestAccount2026이다.
['TestAccount2026', '김민수', '서울대']
***는 ***학교에 재학 중이고 계정 비밀번호는 ***************이다.
